In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import argparse
import yaml
import time
import datetime
import cv2
import numpy as np
import pandas as pd
import random

from skimage import io, transform
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from scipy import ndimage, misc

import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms, utils
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch
from torch.autograd import Variable
import torch.optim as optim
from sklearn.metrics import mean_absolute_error

from google.colab import drive
import os
import warnings
from collections import defaultdict
from sklearn.model_selection import KFold
import time
from scipy import stats
import sys
from sklearn.decomposition import PCA


In [9]:
categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT', 'DIFF_DAYS']

weather_columns = ['sunriseTime', 'sunsetTime', 'moonPhase', 
                     'precipIntensity', 'precipIntensityMax', 'precipProbability', 'temperatureHigh', 'temperatureHighTime',
                     'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime',
                     'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 
                     'windGustTime', 'windBearing', 'cloudCover', 'uvIndex', 'uvIndexTime','visibility', 'temperatureMin',
                     'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin','apparentTemperatureMinTime', 'apparentTemperatureMax',
                     'apparentTemperatureMaxTime','time']

for i in range(1,15):
    for item in weather_columns:
        numerical_columns.append(item + str(i))

print(numerical_columns)

['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR', 'TRAPCOLLECT', 'DIFF_DAYS', 'sunriseTime1', 'sunsetTime1', 'moonPhase1', 'precipIntensity1', 'precipIntensityMax1', 'precipProbability1', 'temperatureHigh1', 'temperatureHighTime1', 'temperatureLow1', 'temperatureLowTime1', 'apparentTemperatureHigh1', 'apparentTemperatureHighTime1', 'apparentTemperatureLow1', 'apparentTemperatureLowTime1', 'dewPoint1', 'humidity1', 'pressure1', 'windSpeed1', 'windGust1', 'windGustTime1', 'windBearing1', 'cloudCover1', 'uvIndex1', 'uvIndexTime1', 'visibility1', 'temperatureMin1', 'temperatureMinTime1', 'temperatureMax1', 'temperatureMaxTime1', 'apparentTemperatureMin1', 'apparentTemperatureMinTime1', 'apparentTemperatureMax1', 'apparentTemperatureMaxTime1', 'time1', 'sunriseTime2', 'sunsetTime2', 'moonPhase2', 'precipIntensity2', 'precipIntensityMax2', 'precipProbability2', 'temperatureHigh2', 'temperatureHighTime2', 'temperatureLow2', 'temperatureLowTime2', 'apparentTemperatureHigh2', 'apparentTemperatureHig

In [0]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/Colab/Mosquito_Berkeley/Approach2/aedes_mosquito_weather_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category').cat.codes

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())

pca_frame = sentinel_frame[numerical_columns + categorical_columns]

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

pca = PCA(n_components=5)
principalComponents = pca.fit_transform(pca_frame)

numerical_columns = ['pc 1', 'pc 2','pc 3', 'pc 4','pc 5']

sentinel_frame_pca = pd.DataFrame(data = principalComponents 
             , columns = numerical_columns)

sentinel_frame_pca["TOTAL"] = sentinel_frame["TOTAL"]

In [11]:
sentinel_frame_pca.columns

Index(['pc 1', 'pc 2', 'pc 3', 'pc 4', 'pc 5', 'TOTAL'], dtype='object')

In [0]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class SwishM(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
        
def create_neural_modules(num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    input_size =  num_numerical_cols
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(num_numerical_cols, output_size, hidden_size, dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_numerical):
        y = self.layers(x_numerical)
        return y

In [0]:
def train_shallow_net(train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr):
    model = Sentinel_net(train_n_t.shape[1], 1, dropout, hidden_size)
    model.train().cuda()

    criterion = torch.nn.L1Loss()
    params_dict = dict(model.named_parameters())
    params = []
    best = sys.maxsize
    r_value = 0
    std_err = 0
    
    for key, value in params_dict.items():
        if 'weight' in key and 'layers' in key:
            params += [{'params':value, 'weight_decay':4e-3}]
        else:
            params += [{'params':value, 'weight_decay':0.0}]

    optimizer = torch.optim.RMSprop(params, lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9, centered=False)
    epoch = 0

    for i in range(1, 1000):
        y_pred = model(train_n_t)
        loss = criterion(y_pred.squeeze(), outputs_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            y_pred2 = model(test_n_t)
            c_out, c_pred = outputs_test.cpu().detach().numpy(), y_pred2.cpu().detach().numpy().squeeze()
            current_score = mean_absolute_error(c_out, c_pred)
            if current_score < best:
                best = current_score
                m, b, r_value, p_value, std_err = stats.linregress(c_out,c_pred)

            model.train()
    
    #print('MAE = ',current_score, 'r2 = ',r_value, 'SE = ',std_err)
    return best, r_value, std_err

In [14]:
start_time = time.time()
learning = [0.01, 0.001] 
best_score_k_fold = sys.maxsize

print('3 Layers Architecture 5 PCA')

for dropout in range(1,5):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame_pca, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame_pca.iloc[train_index], sentinel_frame_pca.iloc[test_index]
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

3 Layers Architecture 5 PCA
MAE =  4.6031537532806395 r2 =  0.19817256532102978 SE =  0.01203642149956645 lr =  0.01 hz =  50 dr =  0.1
Best so far 4.6031537532806395

MAE =  4.549006414413452 r2 =  0.22072022511110764 SE =  0.015034039262660715 lr =  0.001 hz =  50 dr =  0.1
Best so far 4.549006414413452

MAE =  4.6131445407867435 r2 =  0.2383825740989285 SE =  0.010445631192145333 lr =  0.01 hz =  60 dr =  0.1
MAE =  4.6094776630401615 r2 =  0.2363044364906614 SE =  0.014039661179964403 lr =  0.001 hz =  60 dr =  0.1
MAE =  4.6190245151519775 r2 =  0.22386479876699808 SE =  0.010234406309596896 lr =  0.01 hz =  70 dr =  0.1
MAE =  4.581512594223023 r2 =  0.22400913642478532 SE =  0.01583887133996164 lr =  0.001 hz =  70 dr =  0.1
MAE =  4.611732149124146 r2 =  0.23431851721770497 SE =  0.011768705964116296 lr =  0.01 hz =  80 dr =  0.1
MAE =  4.610820770263672 r2 =  0.23646630630479315 SE =  0.016875202373037558 lr =  0.001 hz =  80 dr =  0.1
MAE =  4.599016189575195 r2 =  0.21436861

In [15]:
categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT', 'DIFF_DAYS']

weather_columns = ['sunriseTime', 'sunsetTime', 'moonPhase', 
                     'precipIntensity', 'precipIntensityMax', 'precipProbability', 'temperatureHigh', 'temperatureHighTime',
                     'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime',
                     'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 
                     'windGustTime', 'windBearing', 'cloudCover', 'uvIndex', 'uvIndexTime','visibility', 'temperatureMin',
                     'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin','apparentTemperatureMinTime', 'apparentTemperatureMax',
                     'apparentTemperatureMaxTime','time']

for i in range(1,15):
    for item in weather_columns:
        numerical_columns.append(item + str(i))

print(numerical_columns)

['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR', 'TRAPCOLLECT', 'DIFF_DAYS', 'sunriseTime1', 'sunsetTime1', 'moonPhase1', 'precipIntensity1', 'precipIntensityMax1', 'precipProbability1', 'temperatureHigh1', 'temperatureHighTime1', 'temperatureLow1', 'temperatureLowTime1', 'apparentTemperatureHigh1', 'apparentTemperatureHighTime1', 'apparentTemperatureLow1', 'apparentTemperatureLowTime1', 'dewPoint1', 'humidity1', 'pressure1', 'windSpeed1', 'windGust1', 'windGustTime1', 'windBearing1', 'cloudCover1', 'uvIndex1', 'uvIndexTime1', 'visibility1', 'temperatureMin1', 'temperatureMinTime1', 'temperatureMax1', 'temperatureMaxTime1', 'apparentTemperatureMin1', 'apparentTemperatureMinTime1', 'apparentTemperatureMax1', 'apparentTemperatureMaxTime1', 'time1', 'sunriseTime2', 'sunsetTime2', 'moonPhase2', 'precipIntensity2', 'precipIntensityMax2', 'precipProbability2', 'temperatureHigh2', 'temperatureHighTime2', 'temperatureLow2', 'temperatureLowTime2', 'apparentTemperatureHigh2', 'apparentTemperatureHig

In [0]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/Colab/Mosquito_Berkeley/Approach2/aedes_mosquito_weather_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category').cat.codes

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())

pca_frame = sentinel_frame[numerical_columns + categorical_columns]

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

pca = PCA(n_components=10)
principalComponents = pca.fit_transform(pca_frame)

numerical_columns = ['pc 1', 'pc 2','pc 3', 'pc 4','pc 5','pc 6', 'pc 7','pc 8', 'pc 9','pc 10']

sentinel_frame_pca = pd.DataFrame(data = principalComponents 
             , columns = numerical_columns)

sentinel_frame_pca["TOTAL"] = sentinel_frame["TOTAL"]

In [17]:
start_time = time.time()
learning = [0.01, 0.001] 
best_score_k_fold = sys.maxsize

print('3 Layers Architecture 10 PCA')

for dropout in range(1,5):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame_pca, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame_pca.iloc[train_index], sentinel_frame_pca.iloc[test_index]
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

3 Layers Architecture 10 PCA
MAE =  4.552710247039795 r2 =  0.30014951454610184 SE =  0.014208706442743468 lr =  0.01 hz =  50 dr =  0.1
Best so far 4.552710247039795

MAE =  4.645419931411743 r2 =  0.18563067445597475 SE =  0.012799143407048269 lr =  0.001 hz =  50 dr =  0.1
MAE =  4.514520502090454 r2 =  0.2944454201920871 SE =  0.015223374633944553 lr =  0.01 hz =  60 dr =  0.1
Best so far 4.514520502090454

MAE =  4.666235780715942 r2 =  0.21956609753800355 SE =  0.010375691404109532 lr =  0.001 hz =  60 dr =  0.1
MAE =  4.571209287643432 r2 =  0.21091171684172755 SE =  0.013601694075175516 lr =  0.01 hz =  70 dr =  0.1
MAE =  4.66848406791687 r2 =  0.2335150297506893 SE =  0.00827163288853586 lr =  0.001 hz =  70 dr =  0.1
MAE =  4.544110822677612 r2 =  0.2896585751721109 SE =  0.019822336312651196 lr =  0.01 hz =  80 dr =  0.1
MAE =  4.642506217956543 r2 =  0.20985553347560382 SE =  0.01348409307502077 lr =  0.001 hz =  80 dr =  0.1
MAE =  4.539007711410522 r2 =  0.31082085634600

In [18]:
categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT', 'DIFF_DAYS']

weather_columns = ['sunriseTime', 'sunsetTime', 'moonPhase', 
                     'precipIntensity', 'precipIntensityMax', 'precipProbability', 'temperatureHigh', 'temperatureHighTime',
                     'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime',
                     'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 
                     'windGustTime', 'windBearing', 'cloudCover', 'uvIndex', 'uvIndexTime','visibility', 'temperatureMin',
                     'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin','apparentTemperatureMinTime', 'apparentTemperatureMax',
                     'apparentTemperatureMaxTime','time']

for i in range(1,15):
    for item in weather_columns:
        numerical_columns.append(item + str(i))

print(numerical_columns)

['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR', 'TRAPCOLLECT', 'DIFF_DAYS', 'sunriseTime1', 'sunsetTime1', 'moonPhase1', 'precipIntensity1', 'precipIntensityMax1', 'precipProbability1', 'temperatureHigh1', 'temperatureHighTime1', 'temperatureLow1', 'temperatureLowTime1', 'apparentTemperatureHigh1', 'apparentTemperatureHighTime1', 'apparentTemperatureLow1', 'apparentTemperatureLowTime1', 'dewPoint1', 'humidity1', 'pressure1', 'windSpeed1', 'windGust1', 'windGustTime1', 'windBearing1', 'cloudCover1', 'uvIndex1', 'uvIndexTime1', 'visibility1', 'temperatureMin1', 'temperatureMinTime1', 'temperatureMax1', 'temperatureMaxTime1', 'apparentTemperatureMin1', 'apparentTemperatureMinTime1', 'apparentTemperatureMax1', 'apparentTemperatureMaxTime1', 'time1', 'sunriseTime2', 'sunsetTime2', 'moonPhase2', 'precipIntensity2', 'precipIntensityMax2', 'precipProbability2', 'temperatureHigh2', 'temperatureHighTime2', 'temperatureLow2', 'temperatureLowTime2', 'apparentTemperatureHigh2', 'apparentTemperatureHig

In [0]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/Colab/Mosquito_Berkeley/Approach2/aedes_mosquito_weather_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category').cat.codes

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())

pca_frame = sentinel_frame[numerical_columns + categorical_columns]

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

pca = PCA(n_components=15)
principalComponents = pca.fit_transform(pca_frame)

numerical_columns = ['pc 1', 'pc 2','pc 3', 'pc 4','pc 5','pc 6', 'pc 7','pc 8', 'pc 9','pc 10','pc 11', 'pc 12','pc 13', 'pc 14','pc 15']

sentinel_frame_pca = pd.DataFrame(data = principalComponents 
             , columns = numerical_columns)

sentinel_frame_pca["TOTAL"] = sentinel_frame["TOTAL"]

In [20]:
start_time = time.time()
learning = [0.01, 0.001] 
best_score_k_fold = sys.maxsize

print('3 Layers Architecture 15 PCA')

for dropout in range(1,5):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame_pca, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame_pca.iloc[train_index], sentinel_frame_pca.iloc[test_index]
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

3 Layers Architecture 15 PCA
MAE =  4.480510520935058 r2 =  0.30433667809530957 SE =  0.020700603516798 lr =  0.01 hz =  50 dr =  0.1
Best so far 4.480510520935058

MAE =  4.58159646987915 r2 =  0.25678980368029336 SE =  0.018168905174141464 lr =  0.001 hz =  50 dr =  0.1
MAE =  4.544160652160644 r2 =  0.29620079667868565 SE =  0.019714377459269995 lr =  0.01 hz =  60 dr =  0.1
MAE =  4.587378978729248 r2 =  0.27319185486196973 SE =  0.01604462404913497 lr =  0.001 hz =  60 dr =  0.1
MAE =  4.492531681060791 r2 =  0.39134694236946876 SE =  0.020537720586148916 lr =  0.01 hz =  70 dr =  0.1
MAE =  4.5974987030029295 r2 =  0.27034332342867384 SE =  0.016542662370448068 lr =  0.001 hz =  70 dr =  0.1
MAE =  4.496931695938111 r2 =  0.32401659097711155 SE =  0.023280068564617434 lr =  0.01 hz =  80 dr =  0.1
MAE =  4.614764642715454 r2 =  0.22529942433561895 SE =  0.015988231951641056 lr =  0.001 hz =  80 dr =  0.1
MAE =  4.468846607208252 r2 =  0.2903565612344673 SE =  0.02203271557570349 

In [21]:
categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT', 'DIFF_DAYS']

weather_columns = ['sunriseTime', 'sunsetTime', 'moonPhase', 
                     'precipIntensity', 'precipIntensityMax', 'precipProbability', 'temperatureHigh', 'temperatureHighTime',
                     'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime',
                     'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 
                     'windGustTime', 'windBearing', 'cloudCover', 'uvIndex', 'uvIndexTime','visibility', 'temperatureMin',
                     'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin','apparentTemperatureMinTime', 'apparentTemperatureMax',
                     'apparentTemperatureMaxTime','time']

for i in range(1,15):
    for item in weather_columns:
        numerical_columns.append(item + str(i))

print(numerical_columns)

['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR', 'TRAPCOLLECT', 'DIFF_DAYS', 'sunriseTime1', 'sunsetTime1', 'moonPhase1', 'precipIntensity1', 'precipIntensityMax1', 'precipProbability1', 'temperatureHigh1', 'temperatureHighTime1', 'temperatureLow1', 'temperatureLowTime1', 'apparentTemperatureHigh1', 'apparentTemperatureHighTime1', 'apparentTemperatureLow1', 'apparentTemperatureLowTime1', 'dewPoint1', 'humidity1', 'pressure1', 'windSpeed1', 'windGust1', 'windGustTime1', 'windBearing1', 'cloudCover1', 'uvIndex1', 'uvIndexTime1', 'visibility1', 'temperatureMin1', 'temperatureMinTime1', 'temperatureMax1', 'temperatureMaxTime1', 'apparentTemperatureMin1', 'apparentTemperatureMinTime1', 'apparentTemperatureMax1', 'apparentTemperatureMaxTime1', 'time1', 'sunriseTime2', 'sunsetTime2', 'moonPhase2', 'precipIntensity2', 'precipIntensityMax2', 'precipProbability2', 'temperatureHigh2', 'temperatureHighTime2', 'temperatureLow2', 'temperatureLowTime2', 'apparentTemperatureHigh2', 'apparentTemperatureHig

In [0]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/Colab/Mosquito_Berkeley/Approach2/aedes_mosquito_weather_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category').cat.codes

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())

pca_frame = sentinel_frame[numerical_columns + categorical_columns]

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

pca = PCA(n_components=20)
principalComponents = pca.fit_transform(pca_frame)

numerical_columns = ['pc 1', 'pc 2','pc 3', 'pc 4','pc 5','pc 6', 'pc 7','pc 8', 'pc 9','pc 10','pc 11', 'pc 12','pc 13', 'pc 14','pc 15','pc 16','pc 17', 'pc 18','pc 19', 'pc 20']

sentinel_frame_pca = pd.DataFrame(data = principalComponents 
             , columns = numerical_columns)

sentinel_frame_pca["TOTAL"] = sentinel_frame["TOTAL"]

In [23]:
start_time = time.time()
learning = [0.01, 0.001] 
best_score_k_fold = sys.maxsize

print('3 Layers Architecture 20 PCA')

for dropout in range(1,5):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame_pca, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame_pca.iloc[train_index], sentinel_frame_pca.iloc[test_index]
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

3 Layers Architecture 20 PCA
MAE =  4.538895750045777 r2 =  0.29316860819866297 SE =  0.02162911241279035 lr =  0.01 hz =  50 dr =  0.1
Best so far 4.538895750045777

MAE =  4.532846879959107 r2 =  0.2875506588922424 SE =  0.017491244928436032 lr =  0.001 hz =  50 dr =  0.1
Best so far 4.532846879959107

MAE =  4.47812786102295 r2 =  0.31701809071446563 SE =  0.016791152699574798 lr =  0.01 hz =  60 dr =  0.1
Best so far 4.47812786102295

MAE =  4.536710453033447 r2 =  0.30420218384591047 SE =  0.013683931781673977 lr =  0.001 hz =  60 dr =  0.1
MAE =  4.470712423324585 r2 =  0.3827206799337828 SE =  0.023367968686865845 lr =  0.01 hz =  70 dr =  0.1
Best so far 4.470712423324585

MAE =  4.563828849792481 r2 =  0.26220446082274274 SE =  0.012665931443363412 lr =  0.001 hz =  70 dr =  0.1
MAE =  4.503023529052735 r2 =  0.32779013127402007 SE =  0.019290331269673906 lr =  0.01 hz =  80 dr =  0.1
MAE =  4.606447029113769 r2 =  0.2249912525332287 SE =  0.014800157133459644 lr =  0.001 hz =